## Preprocessing

### Estrazione frasi
Estraiamo un campione di frasi a partire dai file conllu di Wikipedia italiana elaborata. 

In [154]:
import os
import re
import pandas as pd
from tqdm import tqdm


In [155]:
def get_id(line):
    sent_id = re.sub(r'\D', '', line)
    return sent_id

def get_text(line):
    sent = line[9:].rstrip('\n')
    return sent

def get_sentences(file_conllu):
    sentences = []
    ids = []
    with tqdm(total=sum(1 for _ in open(file_conllu, 'r', encoding='utf-8')), desc=f'Progresso estrazione frasi del file {file_conllu}') as pbar:
        for line in open(file_conllu, 'r', encoding='utf-8'):
            pbar.update(1)  # Aggiorna la barra di avanzamento
            if line.startswith("# sent_id"):
                current_id = get_id(line)  
                ids.append(current_id)
            elif line.startswith("# text"):
                current_sent = get_text(line)
                sentences.append(current_sent)
    return ids, sentences

In [156]:
#nel caso in cui dovessimo itereare sui diversi file nella dir
ds_directory = "C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/wiki_conllu"
ds_files = []
for file_name in os.listdir(ds_directory):
    file_path = os.path.join(ds_directory, file_name)
    ds_files.append(file_path)  #ottengo il nome dei diversi file all'interno della directory
print(ds_files)

['C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/wiki_conllu\\wiki_10_2021_015.conllu']


In [157]:
'''with open(ds_files[0], "r", encoding="utf-8") as file:
    # Legge tutte le righe del file in una lista
    righe = file.readlines()'''

'with open(ds_files[0], "r", encoding="utf-8") as file:\n    # Legge tutte le righe del file in una lista\n    righe = file.readlines()'

In [158]:
data_df = pd.DataFrame(columns=["id", "text"])
data_df["id"], data_df["text"] = get_sentences(ds_files[0])   #per ora lavoro su un singolo file

Progresso estrazione frasi del file C:/Users/bergo/OneDrive - University of Pisa/Tesi Magistrale/wiki_conllu\wiki_10_2021_015.conllu: 100%|██████████| 29437395/29437395 [00:54<00:00, 544893.39it/s]


In [159]:
data_df.head()

,id,text
0,1,impeto.
1,2,L’occupazione di Pastrengo.
2,3,Oramai per gli austriaci si trattava di prende...
3,4,Gli uomini della Brigata “Cuneo” intanto entra...
4,5,Ma una batteria di artiglieria a cavallo dei p...


In [160]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057516 entries, 0 to 1057515
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   id      1057516 non-null  object
 1   text    1057516 non-null  object
dtypes: object(2)
memory usage: 16.1+ MB


In [161]:
samplino = data_df.sample(n=6000, random_state=42) #campione piccino per fare le prove

### Estrazione degli indici di leggibilità per ogni frase

Utilizziamo l'API _read-it_ per calcolare gli indici di complessità di ogni frase.

In [162]:
import requests
import sys

In [163]:

SERVER_PATH = "http://api.italianlp.it"
#SERVER_PATH = "http://itanlp-gpu.ilc.cnr.it:13000"

def load_document(text):
    r = requests.post(SERVER_PATH + '/documents/',           # carica il documento nel database del server
                      data={'text': text,                    # durante il caricamento viene eseguita un'analisi linguistica necessaria per calcolare la leggibilità
                          'lang': 'IT',
                          'extra_tasks': ["readability"]     # chiede al server di calcolare anche la leggibilità del docuemnto
                  })
    doc_id = r.json()['id']                                  # id del documento nel database del server, che serve per richiedere i risultati delle analisi
    return doc_id

def get_doc_score(doc_id):
    r = requests.get(SERVER_PATH + '/documents/details/%s' % doc_id)
    result = r.json()
    return result



In [164]:
text = "Ho tanta voglia di pollo del mercato (quello ruspante), non vedo l'ora sia sabato per mangiarlo. Ogni tanto mi chiedo se ne vale la pena spendere 10 euro per mezzo pollo e delle patatine, però quando lo mangio mi sento felice."

doc_id = load_document(text)
r_score = get_doc_score(doc_id)

print(r_score)

{'named_entity_executed': False, 'doc_time': '2024-05-14T16:12:42.100329Z', 'readability_executed': True, 'sentences': {'count': 2, 'prev': None, 'data': [{'sentiment_positive_negative_probability': None, 'sentiment_value': None, 'sequence': 1, 'readability_score_all': 89.3436804643733, 'sentiment_negative_probability': None, 'sentiment_neutral_probability': None, 'tokens': [{'word': 'Ho', 'ten': 'p', 'sequence': 1, 'pos': 'V', 'per': '1', 'lemma': 'avere', 'num': 's', 'named_entity_instance': None, 'dep_type': 'ROOT', 'dep_parent_sequence': None, 'cpos': 'V', 'gen': None, 'mod': 'i'}, {'word': 'tanta', 'ten': None, 'sequence': 2, 'pos': 'DI', 'per': None, 'lemma': 'tanto', 'num': 's', 'named_entity_instance': None, 'dep_type': 'mod', 'dep_parent_sequence': 3, 'cpos': 'D', 'gen': 'f', 'mod': None}, {'word': 'voglia', 'ten': None, 'sequence': 3, 'pos': 'S', 'per': None, 'lemma': 'voglia', 'num': 's', 'named_entity_instance': None, 'dep_type': 'obj', 'dep_parent_sequence': 1, 'cpos': 'S'

In [165]:
print(r_score["sentences"]["data"][0]["raw_text"])

Ho tanta voglia di pollo del mercato (quello ruspante), non vedo l'ora sia sabato per mangiarlo.


In [166]:
#qui iteriamo sui risultati ottenuti per un singolo documento
def get_sen_scores(doc_scores):
    all_sent_rscore = []
    all_sent_text = []
    for i, sent_results in enumerate(doc_scores['sentences']['data']):
        all_sent_rscore.append(sent_results['readability_score_all'])
        all_sent_text.append(sent_results['raw_text'])
    return all_sent_text, all_sent_rscore

#si passa all'api un doc di 1000 frasi per volta.
def readability_extraction(sen_list, x): #x è il numero di frasi che si vogliono calcolare per iterazione 
    all_scores = []
    for i in tqdm(range(0, len(sen_list), x), desc=f"Progresso nell'estrazione degli indici di complessità delle frasi (ogni iterazione corrisponde a {x} frasi)"):
        doc = "\n".join(sen_list[i:i+1000])
        doc_id = load_document(doc)
        r_scores = get_doc_score(doc_id)
        all_scores.append(r_scores)
    return all_scores

#qui si ottiene la struttura dati finale dove per ogni frase abbiamo lo score ottenuto da read_it
def get_final_data(all_scores):
    text_list = []
    score_list = []
    for doc_scores in all_scores:
        text, score = get_sen_scores(doc_scores)
        text_list = text_list + text
        score_list = score_list + score
    return text_list, score_list


In [167]:
all_results = readability_extraction(samplino["text"].tolist(), 2000)
all_sent, all_scores = get_final_data(all_results)

Progresso nell'estrazione degli indici di complessità delle frasi (ogni iterazione corrisponde a 2000 frasi): 100%|██████████| 3/3 [07:01<00:00, 140.61s/it]


### Creazione e valutazione del dataset finale

Si ottiene una struttura dataframe con _id_, _testo e _leggibilità_ per ogni frase. Dopodiché valutiamo le caratteristiche dei dati ottenuti

In [168]:
#si crea un dataframe con tutte le informazioni che ci interessano
data_set = pd.DataFrame(columns=["id", "text", "readability"])
data_set["text"] = all_sent
data_set["readability"] = all_scores
data_set["id"] = range(1, len(data_set) + 1)
data_set.head()

,id,text,readability
0,1,la voce del narratore fa posto a quella dei pe...,34.351971
1,2,Ricardo Bofill nel Taller de Arquitectura usa ...,66.396740
2,3,La denominazione del Gruppo passò da CB a CBOC.,21.319434
3,4,Al traguardo 160 ciclisti portarono a termine ...,4.193945
4,5,"In questa, la fanciulla scappa indossando il m...",94.896129


In [169]:
data_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           3000 non-null   int64  
 1   text         3000 non-null   object 
 2   readability  2323 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 70.4+ KB


Si controlla l'ordine dei dati. Da tenere in considerazione il fatto che "all_results" è in forma di lista contenente i dizionari dei risultati dell'API per ogni "documento" (ovvero ogni x frasi). Quindi l'indice utilizzato per gli elementi di "all_results" per controllare l'ordine sarà sempre compreso tra 0 e x. Mentre quello del nostro dataframe sarà l'equivalente + x*y (dove y, è il numero del documento che stiamo prendendo in considerazione)

In [170]:
#ora controlliamo che l'ordine sia corretto...
tupla = data_set[data_set["id"]==999]
da_result = all_results[0]['sentences']['data'][998] #primo "documento"
print(f"---- Nel dataframe finale ----\n Testo: '{tupla['text'].item()}' - R_score: {tupla['readability'].item()}\n\n") 
print(f"---- Nei risultati ottenuti dall'API ----\n Testo: '{da_result['raw_text']}' - R_score: {da_result['readability_score_all']}")

---- Nel dataframe finale ----
 Testo: 'Federazione di baseball e softball del Niger' - R_score: 92.2796188349079


---- Nei risultati ottenuti dall'API ----
 Testo: 'Federazione di baseball e softball del Niger' - R_score: 92.2796188349079


In [171]:
tupla = data_set[data_set["id"]==1250]
da_result = all_results[1]['sentences']['data'][249] #secondo "documento"
print(f"---- Nel dataframe finale ---- \n Testo: {tupla['text'].item()} - R_score: {tupla['readability'].item()}\n\n") 
print(f"---- Nei risultati ottenuti dall'API ---- \n  Testo: {da_result['raw_text']} - R_score: {da_result['readability_score_all']}")

---- Nel dataframe finale ---- 
 Testo: Nella puntata di "SmackDown" del 14 febbraio gli Ascension sono stati sconfitti dagli American Alpha. - R_score: 6.40928182711793


---- Nei risultati ottenuti dall'API ---- 
  Testo: Nella puntata di "SmackDown" del 14 febbraio gli Ascension sono stati sconfitti dagli American Alpha. - R_score: 6.40928182711793


In [172]:
valori_nulli = data_set[data_set["readability"].isnull()]
valori_nulli #vediamo che le frasi troppo corte non restituiscono un valore di leggibilità. 

,id,text,readability
5,6,Biografia.,NaN
8,9,PCA,NaN
11,12,Partner istituzionali.,NaN
16,17,John Henry Eaton,NaN
19,20,Regio Express,NaN
...,...,...,...
2959,2960,Biografia.,NaN
2962,2963,Matrimonio e figli.,NaN
2969,2970,Discografia.,NaN
2983,2984,Tecniche e manifattura.,NaN
